# **Task 1: Data Ingestion and  Data Preprocessing**

In [ ]:
import os
import sys
import re
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from tenacity import retry, stop_after_attempt, wait_exponential
sys.path.append(os.path.abspath(os.path.join('..')))
from telethon import TelegramClient
from dotenv import load_dotenv
os.getcwd()
rpath = os.path.abspath('../')
if rpath not in sys.path:
    sys.path.insert(0,rpath)

# Load the data

In [7]:
# Load environment variables once
data = pd.read_csv('../data/telegram_data.csv',low_memory=False, index_col=False)

In [8]:
data

,Channel Title,Channel Username,ID,Message,Date,Media Path
0,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14901,NaN,2024-10-09 14:07:16+00:00,photos/@sinayelj_14901.jpg
1,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14900,NaN,2024-10-09 14:07:16+00:00,photos/@sinayelj_14900.jpg
2,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14899,NaN,2024-10-09 14:07:16+00:00,photos/@sinayelj_14899.jpg
3,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14898,mama bag\nኦሪጅናል ማቴሪያል\nበሳይዙ ትልቅ\n 1600 ብር\nFre...,2024-10-09 14:07:16+00:00,photos/@sinayelj_14898.jpg
4,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14897,ኦሪጅናል ማቀፊያ\n1400 ብር\n0905707448\n0909003864\n\...,2024-10-09 14:01:33+00:00,photos/@sinayelj_14897.jpg
...,...,...,...,...,...,...
4651,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,599,NaN,2021-04-16 18:12:10+00:00,photos/@sinayelj_599.jpg
4652,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,598,Baby potty\n0905707448\n0945097042,2021-04-16 18:12:10+00:00,photos/@sinayelj_598.jpg
4653,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,197,NaN,2020-11-26 18:30:52+00:00,NaN
4654,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,182,ውድ የሲና ኪድስ ደምበኞች በድጋሚ ገብቷል \nየመዋኛ ገንዳ ትልቅ ሳይዝ ...,2020-11-10 06:59:31+00:00,photos/@sinayelj_182.jpg


In [9]:
data.dropna()

,Channel Title,Channel Username,ID,Message,Date,Media Path
3,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14898,mama bag\nኦሪጅናል ማቴሪያል\nበሳይዙ ትልቅ\n 1600 ብር\nFre...,2024-10-09 14:07:16+00:00,photos/@sinayelj_14898.jpg
4,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14897,ኦሪጅናል ማቀፊያ\n1400 ብር\n0905707448\n0909003864\n\...,2024-10-09 14:01:33+00:00,photos/@sinayelj_14897.jpg
9,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14892,ልጆች ዳዴ ማለት እንዲሉ የሚለማመዱበት በባትሪ የሚሰራ የልጆች መጫዎቻ ...,2024-10-09 14:00:35+00:00,photos/@sinayelj_14892.jpg
13,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14887,የልጆችን ቀልብ የሚገዛ ውብ ፖፖ\nምቹ የሆነ መነሳት የሚችል ስፖንጅ የሆ...,2024-10-07 12:24:00+00:00,photos/@sinayelj_14887.jpg
19,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14881,baby babi \nየልጆችን ቀልብ የሚገዛ ውብ ፖፖ\nምቹ የሆነ መነሳት ...,2024-10-07 12:17:32+00:00,photos/@sinayelj_14881.jpg
...,...,...,...,...,...,...
4644,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,674,ምቹ የልጆች ፖፖ የሶፍት ማስቀመጫ ያለው 1000ብር\nFree deliver...,2021-04-28 19:12:19+00:00,photos/@sinayelj_674.jpg
4646,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,640,ለልጆች ምቹ እና ኮንፈርተብል ማት \nከላይ ማጫወቻ ያለዉ ከተፈለገ ማጫወ...,2021-04-20 11:16:59+00:00,photos/@sinayelj_640.jpg
4648,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,605,Baby potty \nበባትሪ የሚሰራ ድምፅ ያለው\nየልጆች ፖፖ 899 ብር...,2021-04-16 18:27:22+00:00,photos/@sinayelj_605.jpg
4652,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,598,Baby potty\n0905707448\n0945097042,2021-04-16 18:12:10+00:00,photos/@sinayelj_598.jpg


In [10]:
print("Checking for NaN values in the 'Message' column:")
nan_count = data['Message'].isnull().sum()
print(f"Number of NaN values in 'Message' column: {nan_count}")

Checking for NaN values in the 'Message' column:
Number of NaN values in 'Message' column: 3290


In [11]:
data = data.dropna(subset=['Message'])

# Print the shape of the dataset after dropping NaN values in the "Message" column
print(f"Dataset shape after dropping NaN values in 'Message' column: {data.shape}")

Dataset shape after dropping NaN values in 'Message' column: (1366, 6)


In [12]:
data.head()

,Channel Title,Channel Username,ID,Message,Date,Media Path
3,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14898,mama bag\nኦሪጅናል ማቴሪያል\nበሳይዙ ትልቅ\n 1600 ብር\nFre...,2024-10-09 14:07:16+00:00,photos/@sinayelj_14898.jpg
4,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14897,ኦሪጅናል ማቀፊያ\n1400 ብር\n0905707448\n0909003864\n\...,2024-10-09 14:01:33+00:00,photos/@sinayelj_14897.jpg
9,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14892,ልጆች ዳዴ ማለት እንዲሉ የሚለማመዱበት በባትሪ የሚሰራ የልጆች መጫዎቻ ...,2024-10-09 14:00:35+00:00,photos/@sinayelj_14892.jpg
13,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14887,የልጆችን ቀልብ የሚገዛ ውብ ፖፖ\nምቹ የሆነ መነሳት የሚችል ስፖንጅ የሆ...,2024-10-07 12:24:00+00:00,photos/@sinayelj_14887.jpg
19,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14881,baby babi \nየልጆችን ቀልብ የሚገዛ ውብ ፖፖ\nምቹ የሆነ መነሳት ...,2024-10-07 12:17:32+00:00,photos/@sinayelj_14881.jpg


In [13]:
message=data['Message']
message

3       mama bag\nኦሪጅናል ማቴሪያል\nበሳይዙ ትልቅ\n 1600 ብር\nFre...
4       ኦሪጅናል ማቀፊያ\n1400 ብር\n0905707448\n0909003864\n\...
9       ልጆች ዳዴ ማለት እንዲሉ የሚለማመዱበት በባትሪ የሚሰራ የልጆች መጫዎቻ  ...
13      የልጆችን ቀልብ የሚገዛ ውብ ፖፖ\nምቹ የሆነ መነሳት የሚችል ስፖንጅ የሆ...
19      baby babi \nየልጆችን ቀልብ የሚገዛ ውብ ፖፖ\nምቹ የሆነ መነሳት ...
                              ...                        
4644    ምቹ የልጆች ፖፖ የሶፍት ማስቀመጫ ያለው 1000ብር\nFree deliver...
4646    ለልጆች ምቹ እና ኮንፈርተብል ማት \nከላይ ማጫወቻ ያለዉ ከተፈለገ ማጫወ...
4648    Baby potty \nበባትሪ የሚሰራ ድምፅ ያለው\nየልጆች ፖፖ 899 ብር...
4652                   Baby potty\n0905707448\n0945097042
4654    ውድ የሲና ኪድስ ደምበኞች በድጋሚ ገብቷል \nየመዋኛ ገንዳ ትልቅ ሳይዝ ...
Name: Message, Length: 1366, dtype: object

In [16]:
# Define a function to remove emojis
def remove_emojis(text):
    emoji_pattern = re.compile(
        "[" 
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F700-\U0001F77F"  # alchemical symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"  # Chess Symbols
        "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251" 
        "]+", 
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

# Apply the function to the 'Message' column using .loc
data.loc[:, 'Message'] = data['Message'].apply(remove_emojis)

# Display the updated DataFrame
data.head()

,Channel Title,Channel Username,ID,Message,Date,Media Path
3,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14898,mama bag\nኦሪጅናል ማቴሪያል\nበሳይዙ ትልቅ\n 1600 ብር\nFre...,2024-10-09 14:07:16+00:00,photos/@sinayelj_14898.jpg
4,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14897,ኦሪጅናል ማቀፊያ\n1400 ብር\n0905707448\n0909003864\n\...,2024-10-09 14:01:33+00:00,photos/@sinayelj_14897.jpg
9,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14892,ልጆች ዳዴ ማለት እንዲሉ የሚለማመዱበት በባትሪ የሚሰራ የልጆች መጫዎቻ ...,2024-10-09 14:00:35+00:00,photos/@sinayelj_14892.jpg
13,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14887,የልጆችን ቀልብ የሚገዛ ውብ ፖፖ\nምቹ የሆነ መነሳት የሚችል ስፖንጅ የሆ...,2024-10-07 12:24:00+00:00,photos/@sinayelj_14887.jpg
19,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14881,baby babi \nየልጆችን ቀልብ የሚገዛ ውብ ፖፖ\nምቹ የሆነ መነሳት ...,2024-10-07 12:17:32+00:00,photos/@sinayelj_14881.jpg


In [17]:
data.to_csv('../data/telegram_data_cleaned.csv', index=False)

In [18]:
# Load the cleaned data
data = pd.read_csv('../data/telegram_data_cleaned.csv',low_memory=False, index_col=False)
data

,Channel Title,Channel Username,ID,Message,Date,Media Path
0,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14898,mama bag\nኦሪጅናል ማቴሪያል\nበሳይዙ ትልቅ\n 1600 ብር\nFre...,2024-10-09 14:07:16+00:00,photos/@sinayelj_14898.jpg
1,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14897,ኦሪጅናል ማቀፊያ\n1400 ብር\n0905707448\n0909003864\n\...,2024-10-09 14:01:33+00:00,photos/@sinayelj_14897.jpg
2,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14892,ልጆች ዳዴ ማለት እንዲሉ የሚለማመዱበት በባትሪ የሚሰራ የልጆች መጫዎቻ ...,2024-10-09 14:00:35+00:00,photos/@sinayelj_14892.jpg
3,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14887,የልጆችን ቀልብ የሚገዛ ውብ ፖፖ\nምቹ የሆነ መነሳት የሚችል ስፖንጅ የሆ...,2024-10-07 12:24:00+00:00,photos/@sinayelj_14887.jpg
4,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14881,baby babi \nየልጆችን ቀልብ የሚገዛ ውብ ፖፖ\nምቹ የሆነ መነሳት ...,2024-10-07 12:17:32+00:00,photos/@sinayelj_14881.jpg
...,...,...,...,...,...,...
1361,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,674,ምቹ የልጆች ፖፖ የሶፍት ማስቀመጫ ያለው 1000ብር\nFree deliver...,2021-04-28 19:12:19+00:00,photos/@sinayelj_674.jpg
1362,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,640,ለልጆች ምቹ እና ኮንፈርተብል ማት \nከላይ ማጫወቻ ያለዉ ከተፈለገ ማጫወ...,2021-04-20 11:16:59+00:00,photos/@sinayelj_640.jpg
1363,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,605,Baby potty \nበባትሪ የሚሰራ ድምፅ ያለው\nየልጆች ፖፖ 899 ብር...,2021-04-16 18:27:22+00:00,photos/@sinayelj_605.jpg
1364,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,598,Baby potty\n0905707448\n0945097042,2021-04-16 18:12:10+00:00,photos/@sinayelj_598.jpg


In [20]:
def label_message_utf8_with_birr(message):
    # Split the message at the first occurrence of '\n'
    if '\n' in message:
        first_line, remaining_message = message.split('\n', 1)
    else:
        first_line, remaining_message = message, ""
    
    labeled_tokens = []
    
    # Tokenize the first line
    first_line_tokens = re.findall(r'\S+', first_line)
    
    # Label the first token as B-PRODUCT and the rest as I-PRODUCT
    if first_line_tokens:
        labeled_tokens.append(f"{first_line_tokens[0]} B-PRODUCT")  # First token as B-PRODUCT
        for token in first_line_tokens[1:]:
            labeled_tokens.append(f"{token} I-PRODUCT")  # Remaining tokens as I-PRODUCT
    
    # Process the remaining message normally
    if remaining_message:
        lines = remaining_message.split('\n')
        for line in lines:
            tokens = re.findall(r'\S+', line)  # Tokenize each line while considering non-ASCII characters
            
            for token in tokens:
                # Check if token is a price (e.g., 500 ETB, $100, or ብር)
                if re.match(r'^\d{10,}$', token):
                    labeled_tokens.append(f"{token} O")  # Label as O for "other" or outside of any entity
                elif re.match(r'^\d+(\.\d{1,2})?$', token) or 'ETB' in token or 'ዋጋ' in token or '$' in token or 'ብር' in token:
                    labeled_tokens.append(f"{token} I-PRICE")
                # Check if token could be a location (e.g., cities or general location names)
                elif any(loc in token for loc in ['Addis Ababa', 'ለቡ', 'ለቡ መዳህኒዓለም', 'መገናኛ', 'ቦሌ', 'ሜክሲኮ']):
                    labeled_tokens.append(f"{token} I-LOC")
                # Assume other tokens are part of a product name or general text
                else:
                    labeled_tokens.append(f"{token} O")
    
    return "\n".join(labeled_tokens)

# Apply the updated function to the non-null messages
data['Labeled_Message'] = data['Message'].apply(label_message_utf8_with_birr)

# Display the updated DataFrame
data.head()

,Channel Title,Channel Username,ID,Message,Date,Media Path,Labeled_Message
0,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14898,mama bag\nኦሪጅናል ማቴሪያል\nበሳይዙ ትልቅ\n 1600 ብር\nFre...,2024-10-09 14:07:16+00:00,photos/@sinayelj_14898.jpg,mama B-PRODUCT\nbag I-PRODUCT\nኦሪጅናል O\nማቴሪያል ...
1,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14897,ኦሪጅናል ማቀፊያ\n1400 ብር\n0905707448\n0909003864\n\...,2024-10-09 14:01:33+00:00,photos/@sinayelj_14897.jpg,ኦሪጅናል B-PRODUCT\nማቀፊያ I-PRODUCT\n1400 I-PRICE\...
2,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14892,ልጆች ዳዴ ማለት እንዲሉ የሚለማመዱበት በባትሪ የሚሰራ የልጆች መጫዎቻ ...,2024-10-09 14:00:35+00:00,photos/@sinayelj_14892.jpg,ልጆች B-PRODUCT\nዳዴ I-PRODUCT\nማለት I-PRODUCT\nእን...
3,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14887,የልጆችን ቀልብ የሚገዛ ውብ ፖፖ\nምቹ የሆነ መነሳት የሚችል ስፖንጅ የሆ...,2024-10-07 12:24:00+00:00,photos/@sinayelj_14887.jpg,የልጆችን B-PRODUCT\nቀልብ I-PRODUCT\nየሚገዛ I-PRODUCT...
4,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14881,baby babi \nየልጆችን ቀልብ የሚገዛ ውብ ፖፖ\nምቹ የሆነ መነሳት ...,2024-10-07 12:17:32+00:00,photos/@sinayelj_14881.jpg,baby B-PRODUCT\nbabi I-PRODUCT\nየልጆችን O\nቀልብ O...


In [21]:
# Save the updated labeled dataset to a file in CoNLL format
labeled_data_birr_path = os.path.join(rpath, 'data', 'labeled_telegram_product_price_location.txt')
with open(labeled_data_birr_path, 'w', encoding='utf-8') as f:
    for index, row in data.iterrows():
        f.write(f"{row['Labeled_Message']}\n\n")

In [22]:
print(torch.__version__)

2.4.1+cu121


In [25]:
# Retry decorator
@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=2, max=10))
def load_model():
    tokenizer = AutoTokenizer.from_pretrained("masakhane/afroxlmr-large-ner-masakhaner-1.0_2.0")
    model = AutoModelForTokenClassification.from_pretrained("masakhane/afroxlmr-large-ner-masakhaner-1.0_2.0")
    return tokenizer, model

tokenizer, model = load_model()
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

# Example usage
example = data['Message'][10]
ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-LOC', 'score': np.float32(0.9997359), 'index': 53, 'word': '▁', 'start': 141, 'end': 142}, {'entity': 'B-LOC', 'score': np.float32(0.9829663), 'index': 54, 'word': 'ገር', 'start': 141, 'end': 143}, {'entity': 'I-LOC', 'score': np.float32(0.99756575), 'index': 55, 'word': 'ጂ', 'start': 143, 'end': 144}, {'entity': 'I-LOC', 'score': np.float32(0.9985764), 'index': 56, 'word': '▁ኢ', 'start': 145, 'end': 146}, {'entity': 'I-LOC', 'score': np.float32(0.99717855), 'index': 57, 'word': 'ም', 'start': 146, 'end': 147}, {'entity': 'I-LOC', 'score': np.float32(0.9981951), 'index': 58, 'word': 'ፔ', 'start': 147, 'end': 148}, {'entity': 'I-LOC', 'score': np.float32(0.99899143), 'index': 59, 'word': 'ሪያ', 'start': 148, 'end': 150}, {'entity': 'I-LOC', 'score': np.float32(0.99750596), 'index': 60, 'word': 'ል', 'start': 150, 'end': 151}, {'entity': 'I-LOC', 'score': np.float32(0.9896902), 'index': 61, 'word': '▁ከ', 'start': 152, 'end': 153}, {'entity': 'I-LOC', 'score': np.float32(0.6925

In [27]:
data

,Channel Title,Channel Username,ID,Message,Date,Media Path,Labeled_Message
0,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14898,mama bag\nኦሪጅናል ማቴሪያል\nበሳይዙ ትልቅ\n 1600 ብር\nFre...,2024-10-09 14:07:16+00:00,photos/@sinayelj_14898.jpg,mama B-PRODUCT\nbag I-PRODUCT\nኦሪጅናል O\nማቴሪያል ...
1,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14897,ኦሪጅናል ማቀፊያ\n1400 ብር\n0905707448\n0909003864\n\...,2024-10-09 14:01:33+00:00,photos/@sinayelj_14897.jpg,ኦሪጅናል B-PRODUCT\nማቀፊያ I-PRODUCT\n1400 I-PRICE\...
2,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14892,ልጆች ዳዴ ማለት እንዲሉ የሚለማመዱበት በባትሪ የሚሰራ የልጆች መጫዎቻ ...,2024-10-09 14:00:35+00:00,photos/@sinayelj_14892.jpg,ልጆች B-PRODUCT\nዳዴ I-PRODUCT\nማለት I-PRODUCT\nእን...
3,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14887,የልጆችን ቀልብ የሚገዛ ውብ ፖፖ\nምቹ የሆነ መነሳት የሚችል ስፖንጅ የሆ...,2024-10-07 12:24:00+00:00,photos/@sinayelj_14887.jpg,የልጆችን B-PRODUCT\nቀልብ I-PRODUCT\nየሚገዛ I-PRODUCT...
4,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14881,baby babi \nየልጆችን ቀልብ የሚገዛ ውብ ፖፖ\nምቹ የሆነ መነሳት ...,2024-10-07 12:17:32+00:00,photos/@sinayelj_14881.jpg,baby B-PRODUCT\nbabi I-PRODUCT\nየልጆችን O\nቀልብ O...
...,...,...,...,...,...,...,...
1361,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,674,ምቹ የልጆች ፖፖ የሶፍት ማስቀመጫ ያለው 1000ብር\nFree deliver...,2021-04-28 19:12:19+00:00,photos/@sinayelj_674.jpg,ምቹ B-PRODUCT\nየልጆች I-PRODUCT\nፖፖ I-PRODUCT\nየሶ...
1362,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,640,ለልጆች ምቹ እና ኮንፈርተብል ማት \nከላይ ማጫወቻ ያለዉ ከተፈለገ ማጫወ...,2021-04-20 11:16:59+00:00,photos/@sinayelj_640.jpg,ለልጆች B-PRODUCT\nምቹ I-PRODUCT\nእና I-PRODUCT\nኮን...
1363,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,605,Baby potty \nበባትሪ የሚሰራ ድምፅ ያለው\nየልጆች ፖፖ 899 ብር...,2021-04-16 18:27:22+00:00,photos/@sinayelj_605.jpg,Baby B-PRODUCT\npotty I-PRODUCT\nበባትሪ O\nየሚሰራ ...
1364,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,598,Baby potty\n0905707448\n0945097042,2021-04-16 18:12:10+00:00,photos/@sinayelj_598.jpg,Baby B-PRODUCT\npotty I-PRODUCT\n0905707448 O\...


In [28]:
# Function to check if a string contains Amharic characters
def is_amharic(message):
    return bool(re.search(r'[\u1200-\u137F]', message))

In [29]:
# Function to classify messages
def classify_message(message):
    if pd.isna(message):  # Check for NaN or None
        return 'uncategorized'
    
    if is_amharic(message):
        for category, keywords in categories.items():
            if any(keyword in message for keyword in keywords):
                return category
    else:
        for category, keywords in categories.items():
            if any(keyword in message.lower() for keyword in keywords):
                return category
    return 'uncategorized'

In [30]:
categories = {
    'kids': [
        'toy', 'children', 'kids', 'መጫወቻ', 'play', 'games', 'fun', 'educational', 
        'puzzle', 'doll', 'action figure', 'stuffed animal', 'arts and crafts', 
        'books', 'outdoor toys', 'building blocks', 'baby', 'toddler', 'Baby',
        'መጫወቻዎች','ልጆች'
    ],
    'men': [
        'men', 'grooming', 'shaving', 'beard', 'razor', 'aftershave', 
        'scent', 'deodorant', 'grooming kit', 'haircut', 'fashion', 'suits', 
        'wallet', 'watch', 'accessories', 'fitness', 'shoes', 
        'አስተካክል', 'የብርሃን ዕቃዎች'
    ],
    'women': [
        'women', 'makeup', 'hair dryer', 'lipstick', 'foundation', 'mascara', 
        'skincare', 'nails', 'jewelry', 'dresses', 'handbags', 'accessories', 
        'fashion', 'shoes', 'perfume', 'hairstyle', 'wellness', 'beauty', 'style','Hair Drye',
        'እንቅስቃሴ', 'የፀጉር እቃዎች', 'የውበት እቃዎች'
    ],
    'sport': [
        'gym', 'GYM','fitness', 'exercise', 'እንቅስቃሴ', 'workout', 'training', 'yoga', 
        'running', 'cycling', 'sportswear', 'equipment', 'weights', 'cardio', 
        'aerobics', 'team sports', 'outdoor activities', 'athletics', 'health',  'workout', 'sports',
        'ስፖርት', 'የእንቅስቃሴ መሳሪያዎች'
    ],
    'groceries': [
        'food', 'snacks', 'grocery', 'ምግብ', 'produce', 'fruits', 'vegetables', 
        'meat', 'dairy', 'bread', 'cereal', 'beverages', 'frozen', 'canned', 
        'organic', 'bulk', 'condiments', 'spices', 'snack bars', 'breakfast', 
        'እንቁላል', 'ወተር', 'የምግብ እቃዎች'
    ],
    'accessories': [
        'jewelry', 'bags', 'accessory', 'ቀለበት', 'belts', 'hats', 'scarves', 
        'sunglasses', 'watches', 'hair accessories', 'wallets', 'phone cases', 
        'keychains', 'pins', 'brooches', 'fashion', 'style', 'gifts', 'decor', 'የልብስ መቶከሻ\n\n',
        'የመልክዕ እቃዎች', 'የምታወቅ እቃዎች','Anti-theft ',' Earbuds','PowerBank','Grip Tape','humidifier'
    ],
    'health': [
        'health', 'ጤና', 'wellness', 'nutrition', 'vitamins', 'supplements', 
        'exercise', 'fitness', 'mental health', 'meditation', 'stress relief', 
        'doctor', 'check-up', 'first aid', 'hygiene', 'immune system', 'balance', 
        'self-care', 'አንደኛ ጤና', 'የጤና እቃዎች','pulse'
    ],
    'household': [
        'cleaning', 'furniture', 'decor', 'appliances', 'utensils', 'kitchen', 
        'bathroom', 'laundry', 'storage', 'organization', 'home improvement', 'pan', 
        'gardening', 'tools', 'supplies', 'safety', 'maintenance', 'pets', 'spatulas','Kitchen','Mop',
        'spatulas\n\n','nለኪችንዎ','home', 'comfort', 'ቤት', 'የቤት እቃዎች', 'እንቅስቃሴ','bottle','ፔርሙስ','knife',
        'Glass','የላዛኛ','stove','Ironing Board','Slicer','BLENDER','MULTIFUNCTIONAL BLENDER','Toilet Brush',
        'የቢላ ስብስብ','ቢላ','Oven','fridge', 'መጥበሻ','Toilet','Mob','cookware','Blender','KITCHENWARE','ምንጣፍ','Tablemats'
    ]
}

In [31]:
# Apply classification to the Message column
data['Category'] = data['Message'].apply(classify_message)

# Display the updated DataFrame with categories
print(data[['Message', 'Category']])

                                                Message       Category
0     mama bag\nኦሪጅናል ማቴሪያል\nበሳይዙ ትልቅ\n 1600 ብር\nFre...      household
1     ኦሪጅናል ማቀፊያ\n1400 ብር\n0905707448\n0909003864\n\...      household
2     ልጆች ዳዴ ማለት እንዲሉ የሚለማመዱበት በባትሪ የሚሰራ የልጆች መጫዎቻ  ...           kids
3     የልጆችን ቀልብ የሚገዛ ውብ ፖፖ\nምቹ የሆነ መነሳት የሚችል ስፖንጅ የሆ...           kids
4     baby babi \nየልጆችን ቀልብ የሚገዛ ውብ ፖፖ\nምቹ የሆነ መነሳት ...           kids
...                                                 ...            ...
1361  ምቹ የልጆች ፖፖ የሶፍት ማስቀመጫ ያለው 1000ብር\nFree deliver...           kids
1362  ለልጆች ምቹ እና ኮንፈርተብል ማት \nከላይ ማጫወቻ ያለዉ ከተፈለገ ማጫወ...           kids
1363  Baby potty \nበባትሪ የሚሰራ ድምፅ ያለው\nየልጆች ፖፖ 899 ብር...           kids
1364                 Baby potty\n0905707448\n0945097042           kids
1365  ውድ የሲና ኪድስ ደምበኞች በድጋሚ ገብቷል \nየመዋኛ ገንዳ ትልቅ ሳይዝ ...  uncategorized

[1366 rows x 2 columns]


In [32]:
# Display counts of unique values in the Category column
category_counts = data['Category'].value_counts()
print(category_counts)

Category
kids             575
uncategorized    440
household        309
men               22
groceries         14
health             3
sport              3
Name: count, dtype: int64
